In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import networkx as nx
from pyvis.network import Network

In [2]:
class ID_Generator:
    def __init__(self, max_range = 100):
        self.all_ids = []
        self.max_range = max_range
    
    def generate_id(self):
        new_id = 0
        while(True):
            new_id = np.random.randint(self.max_range)
            if(new_id not in self.all_ids):
                break
        self.all_ids.append(new_id)
        return new_id

class Airport:
    def __init__(self, airport_id):
        self.id = airport_id
        self.airport_info = {}
        self.to_list = []
        self.from_list = []
        self.to_airport_list = []
        self.from_airport_list = []
        
    def __str__(self):
        return self.airport_info['Name']
        
    def init_airport_info(self, airport_info):
        self.airport_info = airport_info
    
    def add_to_list(self, to_route):
        self.to_list.append(to_route)
        self.update_to_airports()
        
    def add_from_list(self, from_route):
        self.from_list.append(from_route)
        self.update_from_airports()
        
    def update_to_airports(self):
        self.to_airport_list = [x.to_airport for x in self.to_list]
    
    def update_from_airports(self):
        self.from_airport_list = [x.from_airport for x in self.from_list]

class Route:
    def __init__(self, route_id):
        self.id = route_id
        self.from_airport = None
        self.to_airport = None
        self.route_info = {}
    
    def __str__(self):
        return str(self.from_airport) + "-" + str(self.to_airport)
    
    def init_from_to(self, from_airport, to_airport):
        self.from_airport = from_airport
        self.to_airport = to_airport
        return self.update_from_to_list()
    
    def init_route_info(self, route_info):
        self.route_info = route_info
    
    def update_from_to_list(self):
        self.to_airport.add_from_list(self)
        self.from_airport.add_to_list(self)
        return self.from_airport, self.to_airport

In [3]:
route_df = pd.DataFrame.from_dict(
    {
        'From':     ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'D', 'D', 'E'],
        'To':       ['B', 'C', 'E', 'A', 'C', 'E', 'A', 'B', 'D', 'A', 'C', 'A'],
        'Traffic':  [ 5,   4,   9,   2,   3,   1,   0,   3,   9,   8,   3,   2 ],
        'Aircraft': ['W', 'W', 'W', 'W', 'W', 'W', 'N', 'W', 'W', 'W', 'N', 'W']
    }
)
route_attr_cols = ['Traffic', 'Aircraft']
route_df

,From,To,Traffic,Aircraft
0,A,B,5,W
1,A,C,4,W
2,A,E,9,W
3,B,A,2,W
4,B,C,3,W
5,B,E,1,W
6,C,A,0,N
7,C,B,3,W
8,C,D,9,W
9,D,A,8,W


In [4]:
airport_df = pd.DataFrame.from_dict(
    {
        'Name':         ['A', 'B', 'C', 'D', 'E'],
        'Airport_Attr': [ 1,   2,   2,   1,   3 ],
        'Tier':         [ 1,   1,   2,   1,   1 ],
        'Population':   [100, 25,  50,  10,  125],
    }
)
airport_attr_cols = ['Name', 'Airport_Attr', 'Tier', 'Population']
airport_df

,Name,Airport_Attr,Tier,Population
0,A,1,1,100
1,B,2,1,25
2,C,2,2,50
3,D,1,1,10
4,E,3,1,125


In [5]:
uniq_airports = [*airport_df['Name'].unique()]
print(uniq_airports)

airport_id_gen = ID_Generator(10)
route_id_gen = ID_Generator(100)

AIRPORTS = {}
ROUTES = {}

for idx, row in airport_df.iterrows():
    airport_obj = Airport(airport_id_gen.generate_id())
    airport_attr = dict([(x, row[x]) for x in airport_attr_cols])
    airport_obj.init_airport_info(airport_attr)
    AIRPORTS[row['Name']] = airport_obj

for idx, row in route_df.iterrows():
    route_obj = Route(route_id_gen.generate_id())
    AIRPORTS[row['From']], AIRPORTS[row['To']] = route_obj.init_from_to(AIRPORTS[row['From']], AIRPORTS[row['To']])
    route_attr = dict([(x, row[x]) for x in route_attr_cols])
    route_obj.init_route_info(route_attr)
    ROUTES[f"{row['From']}-{row['To']}"] = route_obj

print(*AIRPORTS['A'].from_airport_list)
print(*AIRPORTS['A'].to_airport_list)

['A', 'B', 'C', 'D', 'E']
B C D E
B C E


In [6]:
def plot_graph(route_df, airport_df, traffic_var, NODE_COLOR_MAP = None, NODE_SIZE_MAP = None, ROUTE_COLOR_MAP = None, ROUTE_WIDTH_MAP = None, use_node_color = True, use_node_size = True, use_route_color = True, use_route_width = True):
    
    graph = nx.from_pandas_edgelist(route_df, source = 'From', target = 'To', edge_attr = traffic_var, create_using = nx.DiGraph())

    if(use_node_color):
        node_colormap = airport_df.set_index('Name')[[NODE_COLOR_MAP[0]]].to_dict(orient = 'dict')[NODE_COLOR_MAP[0]]
        node_colormap = dict([(x[0], NODE_COLOR_MAP[1](x[1])) for x in node_colormap.items()])
        nx.set_node_attributes(graph, node_colormap, name = 'color')
    
    if(use_node_size):
        node_sizemap = airport_df.set_index('Name')[[NODE_SIZE_MAP[0]]].to_dict(orient = 'dict')[NODE_SIZE_MAP[0]]
        node_sizemap = dict([(x[0], NODE_SIZE_MAP[1](x[1])) for x in node_sizemap.items()])
        nx.set_node_attributes(graph, node_sizemap, name = 'size')

    if(use_route_color):
        route_colormap = route_df.set_index(['From', 'To'])[[ROUTE_COLOR_MAP[0]]].to_dict(orient = 'dict')[ROUTE_COLOR_MAP[0]]
        route_colormap = dict([((x[0][0], x[0][1]), ROUTE_COLOR_MAP[1](x[1])) for x in route_colormap.items()])
        nx.set_edge_attributes(graph, route_colormap, name = 'color')

    if(use_route_width):
        route_widthmap = route_df.set_index(['From', 'To'])[[ROUTE_WIDTH_MAP[0]]].to_dict(orient = 'dict')[ROUTE_WIDTH_MAP[0]]
        route_widthmap = dict([((x[0][0], x[0][1]), ROUTE_WIDTH_MAP[1](x[1])) for x in route_widthmap.items()])
        nx.set_edge_attributes(graph, route_widthmap, name = 'weight')

    # nx.draw_networkx(graph, node_color = colormap.values())
    net = Network(notebook = True, directed = True, filter_menu = True, select_menu = True, height="750px", width="100%", cdn_resources='remote')
    net.from_nx(graph)
    return net

In [7]:
# Input variables
def node_attr_to_color_map(attr):
    if(attr == 1):
        return 'blue'
    else:
        return 'yellow'

def node_attr_to_size_map(attr):
    return attr/5

def route_attr_to_color_map(attr):
    if(attr == 'W'):
        return 'red'
    else:
        return 'green'
    
def route_attr_to_width_map(attr):
    return attr/2

NODE_COLOR_MAP = ['Tier', node_attr_to_color_map]
NODE_SIZE_MAP = ['Population', node_attr_to_size_map]
ROUTE_COLOR_MAP = ['Aircraft', route_attr_to_color_map]
ROUTE_WIDTH_MAP = ['Traffic', route_attr_to_width_map]

In [8]:
# net = plot_graph(route_df, airport_df, "Traffic", NODE_COLOR_MAP = NODE_COLOR_MAP, use_node_size = False, ROUTE_COLOR_MAP = ROUTE_COLOR_MAP, ROUTE_WIDTH_MAP = ROUTE_WIDTH_MAP)
# net.show('graph_network_sample.html')

In [9]:
route_df = pd.read_csv("../PreProcessed_Datasets/AirRouteDatasets/FlightConnectionsData_Flights.csv")
route_attr_cols = ['Distance', 'Time', 'Cheapest Price', 'Aircraft Type', 'Number of Flights']
print(route_df.shape)
route_df.tail()

(189, 7)


,From,To,Distance,Time,Cheapest Price,Aircraft Type,Number of Flights
184,BLR,VNS,910,2h25m,85,Others,2.0
185,BOM,IXJ,947,2h50m,111,Others,1.0
186,BOM,BBI,845,2h20m,46,Others,1.0
187,BOM,BLR,517,1h30m,40,Others,1.0
188,BOM,GOI,263,1h15m,25,Others,1.0


In [10]:
airport_df = pd.read_csv("../PreProcessed_Datasets/AirRouteDatasets/FlightConnectionsData_Airports.csv")
airport_attr_cols = ['City/Town', 'Airport Traffic Level']
airport_df.head()

,City/Town,Name,Airport Traffic Level
0,Ahmedabad,AMD,1
1,Bhuj,BHJ,3
2,Jaisalmer,JSA,3
3,Jodhpur,JDH,2
4,Jammu,IXJ,3


In [11]:
uniq_airports = [*airport_df['Name'].unique()]
print("Unique Airports:")
print(uniq_airports)

airport_id_gen = ID_Generator(100)
route_id_gen = ID_Generator(1000)

AIRPORTS = {}
ROUTES = {}

for idx, row in airport_df.iterrows():
    airport_obj = Airport(airport_id_gen.generate_id())
    airport_attr = dict([(x, row[x]) for x in airport_attr_cols])
    airport_obj.init_airport_info(airport_attr)
    AIRPORTS[row['Name']] = airport_obj

for idx, row in route_df.iterrows():
    route_obj = Route(route_id_gen.generate_id())
    AIRPORTS[row['From']], AIRPORTS[row['To']] = route_obj.init_from_to(AIRPORTS[row['From']], AIRPORTS[row['To']])
    route_attr = dict([(x, row[x]) for x in route_attr_cols])
    route_obj.init_route_info(route_attr)
    ROUTES[f"{row['From']}-{row['To']}"] = route_obj

print("Number of Airports:")
print(len(AIRPORTS))
print("Number of Flights:")
print(len(ROUTES))

print(len(AIRPORTS['AMD'].from_airport_list))
print(len(AIRPORTS['AMD'].to_airport_list))

Unique Airports:
['AMD', 'BHJ', 'JSA', 'JDH', 'IXJ', 'ATQ', 'IXC', 'DED', 'DEL', 'JAI', 'AGR', 'LKO', 'IXB', 'VNS', 'PAT', 'IXR', 'CCU', 'IDR', 'BHO', 'BBI', 'RPR', 'NAG', 'HYD', 'MAA', 'BLR', 'COK', 'ISK', 'PNQ', 'KLH', 'IXG', 'GOX', 'GOI', 'BOM', 'IXM', 'RAJ', 'STV', 'DBR', 'RDP', 'BDQ', 'PNY', 'HBX', 'IXE', 'VGA', 'CNN', 'TCR', 'IXZ', 'KJB', 'IXA', 'TIR', 'CJB', 'KNU', 'GBI', 'VTZ', 'CCJ', 'TRV', 'IXX', 'IXD', 'RJA', 'TRZ', 'SAG', 'GAU', 'GWL', 'UDR', 'GOP', 'SDW', 'JGA', 'SXR', 'DIU', 'BHU', 'IXU', 'JLR', 'IXY', 'IXL', 'BEK']
Number of Airports:
74
Number of Flights:
168
4
31


In [12]:
# Input variables
def node_attr_to_color_map(attr):
    if(attr == 1):
        return 'green'
    elif(attr == 2):
        return 'yellow'
    else:
        return 'red'

def route_attr_to_color_map(attr):
    if(attr == 'Narrow Body'):
        return 'yellow'
    elif(attr == 'Turbo-prop'):
        return 'red'
    elif(attr == 'Regional Jet'):
        return 'green'
    else:
        return 'black'
    
def route_attr_to_width_map(attr):
    return attr / 200

NODE_COLOR_MAP = ['Airport Traffic Level', node_attr_to_color_map]
ROUTE_COLOR_MAP = ['Aircraft Type', route_attr_to_color_map]
ROUTE_WIDTH_MAP = ['Distance', route_attr_to_width_map]

In [14]:
net = plot_graph(route_df, airport_df, "Number of Flights", NODE_COLOR_MAP = NODE_COLOR_MAP, use_node_size = False, ROUTE_COLOR_MAP = ROUTE_COLOR_MAP, use_route_width = False)
net.show('graph_network_actual.html')